Viena užduotis maždaug
vienai paskaitai. Techniniai reikalavimai –
Jupyter Notebook’as arba Streamlit aplinka. Grafikai turi turėti
pavadinimus, ašys pavadintos prasmingai, jei grafike yra daugiau nei
viena kreivė – legenda būtina. Panaudoti boxplot,
swarmplot, heatmap, pairplot, (h)bar,
histogram, scatter. Būtina pasirašyti 1 – 2
sakinius į JupyterNotebook’ą apie tai, ką matote, ką galite
pasakyti iš grafiko (ką matote grafike, 1-2 sakiniai vienam
grafikui, tad jei turėsite 10 grafikų – 20 sakinių).

5  

Duomenys:
https://data.gov.lt/datasets/2829/
- Stacionaro ir ambulatoriniai ligoniai, gydyti nuo traumų ir
apsinuodijimų. Įvertinkite patiriamų traumų, apsinuodijimų
dažnius/pasiskirstymą, sunkumą pagal amžių/amžiaus
grupes, lytį, paros, metų, darbo dienų/švenčių/išeiginių. Ar
traumų pobūdis/dažnis, apsinuodijimų kiekis, pobūdis, sunkumas
priklauso nuo užfiksuotų meteorologinių sąlygų, mėnulio fazės,
švenčių/laisvadienių?
Duomenys:
https://data.gov.lt/datasets/2793/#info
– PAGD iškvietimai; https://data.gov.lt/datasets/1667/#info
– Policijos
iškvietimai. Ar
iškvietimų dažnis/kiekis priklauso nuo paros,
metų, darbo dienų/švenčių/išeiginių, mėnulio
fazės, švenčių/laisvadienių?
Mėnulio
fazės turi būti pasiekiamos per http užklausas – iš esmės
pasinaudojate web scrapingu, kad gautumėte norimą informaciją.
Papildoma informacija -
https://ligoniukasa.lrv.lt/lt/veiklos-sritys/informacija-gyventojams/gydymo-ir-sveikatos-prieziuros-paslaugos/stebejimo-paslaugos/
;

TLK kodų paaiškinimai yra šiame excel faile:
https://ligoniukasa.lrv.lt/uploads/ligoniukasa/documents/files/1(9).xlsx

Jūs turėsite sujungti tris failus:
https://data.gov.lt/datasets/2829/data/Pacientas/
https://data.gov.lt/datasets/2829/data/Ligonine/
ir pateiktą excel'io failą, kad turėtumėte vientisą lentelę su reikiamais duomenimis.
Prie jų jungsite infromaciją iš
https://astropixels.com/ephemeris/phasescat/phasescat.html
ARBA day.lt - bus paprasčiau:
https://day.lt/diena/2021.04.22    <--- adreso laukelyje įrašius norimą datą, gaunama informacija apie tą dieną.

iš kurios, naudodami web scraping'ą, išgausite reikiamą informaciją apie fazes. (requests bibliotekos užteks)

Kuriu ir testuoju funkciją, nustatančią mėnulio fazę.

In [49]:
import ephem

def get_moon_phase(date=None):
    moon = ephem.Moon()
    moon.compute(date)
    phase = moon.phase
    return phase

# Naudojimas:
date = '2025/02/05'  # Data 'YYYY/MM/DD'
print(f"Mėnulio fazė {date}: {get_moon_phase(date):2.0f}%")

date = '2008/08/01'  # Jaunatis
print(f"Mėnulio fazė {date}: {get_moon_phase(iso_date):3.0f}% (Jaunatis)")

date = '2001/11/01'  # Pilnatis
print(f"Mėnulio fazė {date}: {get_moon_phase(date):2.0f}%")


Mėnulio fazė 2025/02/05: 46%
Mėnulio fazė 2008/08/01:   0% (Jaunatis)
Mėnulio fazė 2001/11/01: 100%


In [50]:
from dateutil import parser
import ephem

def get_moon_phase(iso_date=None):
    # Konvertuojama ISO data į datetime objektą
    date = parser.parse(iso_date)
    # Sukuriama `ephem` mėnulio objektas
    moon = ephem.Moon()
    # Data pateikiama `ephem` suprantamu formatu
    moon.compute(date)
    phase = moon.phase
    return phase

# Naudojimas:
date = '2025-02-05'  # Data
print(f"Mėnulio fazė {date}: {get_moon_phase(date):2.0f}%")

date = '2001-11-01'  # Pilnatis
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}% (Pilnatis)")

date = '2008-08-01'  # Jaunatis
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}% (Jaunatis)")


Mėnulio fazė 2025-02-05: 46%
Mėnulio fazė 2001-11-01: 100% (Pilnatis)
Mėnulio fazė 2008-08-01:   0% (Jaunatis)


In [56]:
from dateutil import parser
import ephem
from datetime import datetime

def get_moon_phase(iso_date=None):
    # Jei data nenurodyta, naudojama šiandienos data
    if not iso_date:
        date = datetime.now()
        # iso_date.replace(hour=0, minute=0, second=0, microsecond=0)
    else:
        # Konvertuojama ISO data į datetime objektą
        date = parser.parse(iso_date)

    # Sukuriama `ephem` mėnulio objektas
    moon = ephem.Moon()
    # Data pateikiama `ephem` suprantamu formatu
    moon.compute(date)
    phase = moon.phase
    # print(f"({date}, {phase})")
    return phase

# Naudojimas:
print(f"Mėnulio fazė šiuo metu: {get_moon_phase():3.0f}%")

date = '2025-02-05 19:27'  # Data
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}%")

date = '2001-11-01'  # Pilnatis
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}% (Pilnatis)")

date = '2008-08-01'  # Jaunatis
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}% (Jaunatis)")


Mėnulio fazė šiuo metu:  55%
Mėnulio fazė 2025-02-05 19:27:  55%
Mėnulio fazė 2001-11-01: 100% (Pilnatis)
Mėnulio fazė 2008-08-01:   0% (Jaunatis)


In [59]:
from datetime import datetime
import ephem

def get_moon_phase(date=None):
    if date is None:
        date = datetime.now()
    else:
        date = datetime.strptime(date, "%Y-%m-%d")
    
    moon = ephem.Moon()
    # Konvertuojama 'datetime' objektas į 'ephem' naudojamą formatą
    moon.compute(date.strftime('%Y/%m/%d')) # date.strftime('%Y/%m/%d')
    phase = moon.phase
    return phase

# Naudojimas:
print(f"Mėnulio fazė šiuo metu: {get_moon_phase():3.0f}%")

date = '2025-02-05'  # Data
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}%")

date = '2001-11-01'  # Pilnatis
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}% (Pilnatis)")

date = '2008-08-01'  # Jaunatis
print(f"Mėnulio fazė {date}: {get_moon_phase(date):3.0f}% (Jaunatis)")


Mėnulio fazė šiuo metu:  46%
Mėnulio fazė 2025-02-05:  46%
Mėnulio fazė 2001-11-01: 100% (Pilnatis)
Mėnulio fazė 2008-08-01:   0% (Jaunatis)
